In [ ]:
import math

def make_prime(number, *exclude):
    while True:
        for i in range(2, number):
            if number in exclude:
                number += 1
            if number % i == 0:
                number += 1
                break
        if i == number - 1:
            break
    return number

def get_e(p, q, phi_n):
    for i in range(2, phi_n):
        if math.gcd(i, phi_n) == 1 and i != p and i != q:
            return i

def eem(phi_n, e):
    r = [phi_n, e]
    qt = []
    x = [1, 0]
    y = [0, 1]

    while r[-1] != 1:
        qt.append(r[-2] // r[-1])
        r.append(r[-2] % r[-1])
        x.append(x[-2] - qt[-1] * x[-1])
        y.append(y[-2] - qt[-1] * y[-1])

    print('r\tqt\tx\ty')
    for i in range(len(r)):
        print(f'{r[i]}\t{qt[i] if i < len(qt) else "-"}\t{x[i]}\t{y[i]}')

    return y[-1] % phi_n

p = len('abcdefghijklmnopqrstuvwxyz')
q = len('abcd')

p = make_prime(p)
q = make_prime(q, p)

n = p * q
phi_n = (p - 1) * (q - 1)
e = get_e(p, q, phi_n)
d = eem(phi_n, e)
print('P Value:', p)
print('Q Value:' , q)
print('N Value:', n)
print('D Value:' , d)
print('E Value:', e)

print()
print('Public key:', (e, n))
print('Private key:', (d, n))
print()

pt = int(input("Enter plain text (int): "))

ct = pow(pt, e, n)
print("After encryption\nThe cipher text is:", ct, '\n')

pt = pow(ct, d, n)
print('After decryption\nThe plain text is:', pt)

r	qt	x	y
112	37	1	0
3	-	0	1
1	-	1	-37
P Value: 29
Q Value: 5
N Value: 145
D Value: 75
E Value: 3

Public key: (3, 145)
Private key: (75, 145)

Enter plain text (int): 20
After encryption
The cipher text is: 25 

After decryption
The plain text is: 20


In [ ]:
%pip install cryptography

In [ ]:
import random


def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a


def multiplicative_inverse(e, phi):
    d = 0
    x1 = 0
    x2 = 1
    y1 = 1
    temp_phi = phi

    while e > 0:
        temp1 = temp_phi//e
        temp2 = temp_phi - temp1 * e
        temp_phi = e
        e = temp2

        x = x2 - temp1 * x1
        y = d - temp1 * y1

        x2 = x1
        x1 = x
        d = y1
        y1 = y

    if temp_phi == 1:
        return d + phi


'''
Tests to see if a number is prime.
'''


def is_prime(num):
    if num == 2:
        return True
    if num < 2 or num % 2 == 0:
        return False
    for n in range(3, int(num**0.5)+2, 2):
        if num % n == 0:
            return False
    return True


def generate_key_pair(p, q):
    if not (is_prime(p) and is_prime(q)):
        raise ValueError('Both numbers must be prime.')
    elif p == q:
        raise ValueError('p and q cannot be equal')

    n = p * q

    phi = (p-1) * (q-1)

    e = random.randrange(1, phi)

    g = gcd(e, phi)
    while g != 1:
        e = random.randrange(1, phi)
        g = gcd(e, phi)
    d = multiplicative_inverse(e, phi)

    return ((e, n), (d, n))


def encrypt(pk, plaintext):
    key, n = pk
    cipher = [pow(ord(char), key, n) for char in plaintext]
    return cipher


def decrypt(pk, ciphertext):
    key, n = pk
    aux = [str(pow(char, key, n)) for char in ciphertext]
    plain = [chr(int(char2)) for char2 in aux]
    return ''.join(plain)


if __name__ == '__main__':
    p = int(input(" - Enter a prime number (17, 19, 23, etc): "))
    q = int(input(" - Enter another prime number (Not one you entered above): "))

    print(" - Generating your public / private key-pairs now")

    public, private = generate_key_pair(p, q)

    print(" - Your public key is ", public, " and your private key is ", private)

    message = input(" - Enter a message to encrypt with your public key: ")
    encrypted_msg = encrypt(public, message)

    print(" - Your encrypted message is: ", ''.join(map(lambda x: str(x), encrypted_msg)))
    print(" - Decrypting message with private key ", private,)
    print(" - Your message is: ", decrypt(private, encrypted_msg))

 - Enter a prime number (17, 19, 23, etc): 17
 - Enter another prime number (Not one you entered above): 19
 - Generating your public / private key-pairs now
 - Your public key is  (229, 323)  and your private key is  (205, 323)
 - Enter a message to encrypt with your public key: Hello World
 - Your encrypted message is:  276118262262195534919519026236
 - Decrypting message with private key  (205, 323)
 - Your message is:  Hello World
